In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd

sys.path.append("../")
import circletrack_behavior as ctb
import plotting_functions as pf

def set_groups(df):
    df['group'] = df['mouse']
    df['group'] = df['group'].replace({'mca01': 'wt_a', 'mca02': 'wt_a', 'mca03': 'wt_a', 'mca04': 'tg_a', 'mca05': 'tg_a', 'mca06': 'tg_a',
                                       'mca07': 'tg_a', 'mca08': 'tg_a', 'mca09': 'tg_a', 'mca10': 'wt_a', 'mca11': 'wt_a', 'mca12': 'wt_a',
                                       'mca13': 'tg_y', 'mca14': 'tg_y', 'mca15': 'tg_y', 'mca16': 'tg_y', 'mca17': 'tg_y', 'mca18': 'tg_y',
                                       'mca19': 'wt_y', 'mca20': 'wt_y', 'mca21': 'wt_y', 'mca22': 'wt_y', 'mca23': 'wt_y'})
    return df

In [ ]:
## Set path variables
behavior_type = 'circletrack_data'
csv_name = 'circle_track.csv'
dpath = os.path.abspath('../../MultiCon_Behavior/MultiCon_Aging1/{}/data/**/**/**/{}'.format(behavior_type, csv_name))
group_colors = ['green', 'blue', 'turquoise', 'darkgrey'] ## tg_a, tg_y, wt_a, wt_y
## Create file list dataframe
file_list = ctb.get_file_list(dpath)
mouse_id = []
for file in file_list:
    mouse_id.append(ctb.get_mouse(file, str2match='(mca[0-9]+)'))
combined = ctb.combine(file_list, mouse_id)

### Get lick accuracy for a session.

In [ ]:
lick_df = pd.DataFrame()
for mouse in np.unique(mouse_id):
    lick_data = ctb.get_lick_accuracy(combined, mouse)
    lick_df = pd.concat([lick_df, lick_data], ignore_index=True)
lick_df = set_groups(lick_df)

In [ ]:
lick_df[lick_df['mouse'] == 'mca04']

In [ ]:
## Lick accuracy
fig = pf.plot_behavior_across_days(lick_df, x_var='day', y_var='percent_correct', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=group_colors, avg_color='darkgrey',
                                   x_title='Day', y_title='Lick Accuracy (%)', titles=['Circle Track: Lick Accuracy'], height=800, width=800)
fig.show()

### Get total rewards for a session.

In [ ]:
reward_df = pd.DataFrame()
for mouse in np.unique(mouse_id):
    reward_data = ctb.get_total_rewards(combined, mouse)
    reward_df = pd.concat([reward_df, reward_data], ignore_index=True)
reward_df = set_groups(reward_df)

In [ ]:
fig = pf.plot_behavior_across_days(reward_df, x_var='day', y_var='total_rewards', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=group_colors, avg_color='darkgrey', expert_line=False, chance=False,
                                   x_title='Day', y_title='Total Rewards', titles=['Circle Track: Rewards'], height=800, width=800)
fig.show()

### Set up mouse maze list for the day.

In [ ]:
day = 'Day 17' ## edit depending on day
retired_mice = ['mca04'] ## 'mca09', 'mca18' ## add retired mouse to this list
maze_info = pd.read_csv('../../MultiCon_Behavior/MultiCon_Aging1/maze_yml/maze_info.csv')
todays_mazes = pd.read_csv('../../MultiCon_Behavior/MultiCon_Aging1/maze_yml/todays_mazes.csv') ## download at end of the day
run_dict = {'maze1': [], 'maze2': [], 'maze3': [], 'maze4': []}
letter_dict = {'maze1': [], 'maze2': [], 'maze3': [], 'maze4': []}
for mouse in np.unique(reward_df['mouse']):
    if mouse in retired_mice:
        pass
    else:
        maze_letter = todays_mazes.loc[todays_mazes['Mouse'] == mouse, day].reset_index(drop=True)

        if maze_letter[0] == 'A':
            maze_number = maze_info.loc[maze_info['Mouse'] == mouse, "if 'A'"].reset_index(drop=True)
        elif maze_letter[0] == 'B':
            maze_number = maze_info.loc[maze_info['Mouse'] == mouse, "if 'B'"].reset_index(drop=True)
        elif maze_letter[0] == 'C':
            maze_number = maze_info.loc[maze_info['Mouse'] == mouse, "if 'C'"].reset_index(drop=True)
        elif maze_letter[0] == 'D':
            maze_number = maze_info.loc[maze_info['Mouse'] == mouse, "if 'D'"].reset_index(drop=True)

        if maze_number[0] == 1:
            run_dict['maze1'].append(mouse)
            letter_dict['maze1'].append(maze_letter[0])
        elif maze_number[0] == 2:
            run_dict['maze2'].append(mouse)
            letter_dict['maze2'].append(maze_letter[0])
        elif maze_number[0] == 3:
            run_dict['maze3'].append(mouse)
            letter_dict['maze3'].append(maze_letter[0])
        elif maze_number[0] == 4:
            run_dict['maze4'].append(mouse)
            letter_dict['maze4'].append(maze_letter[0])
## Convert to dataframe to paste into excel         
run_df = pd.DataFrame.from_dict(run_dict, orient='index').T 
letter_df = pd.DataFrame.from_dict(letter_dict, orient='index').T

In [ ]:
run_df

In [ ]:
letter_df